# Calculating Hellmann-Feynman Forces

This code shows how to calculate Hellmann-Feynman forces from an ML electron density. This requires a custom-built version of psi4 available at https://github.com/JoshRackers/psi4. It will also show how to compute long-range versions of the Hellmann-Feynman force.

In [ ]:
import sys
if "/home/jracker/codes/psi4/build/stage/lib" not in sys.path:
    sys.path.append("/home/jracker/codes/psi4/build/stage/lib")
    #print(sys.path)
import psi4

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os
import math
import numpy as np

import torch
import torch_geometric

sys.path.append(os.path.dirname(os.path.abspath('')))
from utils import get_iso_permuted_dataset
from utils import flatten_list
from utils import compute_potential_field

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
print (device)

# conversion from Hartrees to kcal/mol
ha2kcalmol = 627.5094740631

In [ ]:
# first, get dataset
hhh = "../data/h_s_only_def2-universal-jfit-decontract_density.out"
ooo = "../data/o_s_only_def2-universal-jfit-decontract_density.out"

fours = "../tests/test_data_generation/testdata_w4.pkl"
w04_dataset = get_iso_permuted_dataset(fours,o_iso=ooo,h_iso=hhh)

data_loader = torch_geometric.data.DataLoader(w04_dataset[:], batch_size=1, shuffle=False)

In [ ]:
# now get model
from e3nn.nn.models.gate_points_2101 import Network
from e3nn import o3

model_kwargs = {
        "irreps_in": "2x 0e", #irreps_in 
        "irreps_hidden": [(mul, (l, p)) for l, mul in enumerate([125,40,25,15]) for p in [-1, 1]], #irreps_hidden
        #"irreps_hidden": "100x0e + 100x0o",
        "irreps_out": "12x0e + 5x1o + 4x2e + 2x3o + 1x4e", #irreps_out
        "irreps_node_attr": None, #irreps_node_attr
        "irreps_edge_attr": o3.Irreps.spherical_harmonics(3), #irreps_edge_attr
        "layers": 3,
        "max_radius": 3.5,
        "number_of_basis": 10,
        "radial_layers": 1,
        "radial_neurons": 128,
        "num_neighbors": 12.2298,
        "num_nodes": 24,
        "reduce_output": False,
    }

model = Network(**model_kwargs)
model.to(device)

model.load_state_dict(torch.load('pretrained_model.pt'))

In [ ]:
# now compute hellmann-feynman forces

# get first structure
data = data_loader.dataset[0]

# inference on model
mask = torch.where(data.y == 0, torch.zeros_like(data.y), torch.ones_like(data.y)).detach()
y_ml = model(data.to(device))*mask.to(device)

# get atomic positions
x_atoms = data.pos_orig[:,0].cpu().detach().flatten()
y_atoms = data.pos_orig[:,1].cpu().detach().flatten()
z_atoms = data.pos_orig[:,2].cpu().detach().flatten()
Rs = [(12, 0), (5, 1), (4, 2), (2, 3), (1, 4)]

# evaluate full electrostatic field at atomic positions
t_pot, t_field, m_pot, m_field = compute_potential_field(x_atoms,y_atoms,z_atoms,data,y_ml.detach(),Rs,intermolecular=False)

# now compute forces
charges = data.z.cpu().detach().numpy()
target_forces = t_field*charges
ml_forces = m_field*charges

print("Force error (Ha/bohr)")
print(ml_forces-target_forces)

In [ ]:
# evaluate long-range electrostatic field at atomic positions
# exclude contributions from <7 Å interactions
# NOTE: for small clusters like this, this excludes all interactions
#       but for larger clusters, this is significant
lr_t_pot, lr_t_field, lr_m_pot, lr_m_field = compute_potential_field(x_atoms,y_atoms,z_atoms,data,y_ml.detach(),Rs,intermolecular=True,rad=13.2281)

# now compute forces
lr_target_forces = lr_t_field*charges
lr_ml_forces = lr_m_field*charges

print("Force error (Ha/bohr)")
print(lr_ml_forces-lr_target_forces)